In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_groq import ChatGroq
from langchain.schema import Document
import os

In [10]:
class RAGAnswerGenerator:
    def __init__(self, my_dir):
        os.environ["GROQ_API_KEY"] = "your_groq_api_key"
        
        self.my_dir = my_dir
        
        self.embeddings = HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-mpnet-base-v2',
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        
        self.vectorstore = self._create_vectorstore()
        
        self.llm = ChatGroq(
            model_name="mixtral-8x7b-32768", 
            temperature=0.1,  
            max_tokens=1024
        )
        
        self.qa_chain = self._create_qa_chain()

    def _create_vectorstore(self):
        documents = []
        for filename in os.listdir(self.my_dir):
            if filename.endswith('.txt'):
                filepath = os.path.join(self.my_dir, filename)
                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        text = file.read()
                        documents.append(Document(
                            page_content=text, 
                            metadata={"source": filename}
                        ))
                except Exception as e:
                    print(f"Error reading file {filename}: {e}")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,  
            chunk_overlap=100,
            separators=["\n\n", "\n", " ", ""]
        )
        texts = text_splitter.split_documents(documents)
        
        vectorstore = FAISS.from_documents(texts, self.embeddings)
        return vectorstore

    def _create_qa_chain(self):
        return RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_kwargs={
                    "k": 3, 
                    "fetch_k": 10  
                }
            ),
            return_source_documents=True
        )

    def generate_answer(self, question):
    
        result = self.qa_chain({"question": question})
        
        answer = result.get('answer', 'I could not find a specific answer.')
        sources = result.get('source_documents', [])
        
        source_info = "\n\nSources:" + ''.join([f"\n- {doc.metadata.get('source', 'Unknown source')}" for doc in sources])
        
        return answer + source_info

In [11]:
class RAGChatPipeline:
    def __init__(self, my_dir):
        self.my_dir = my_dir
        self.rag_generator = RAGAnswerGenerator(my_dir)

    def run_chat(self):
        print("Starting the RAG Chat System...")
        print("Type 'exit' to quit the chat.")
        
        while True:
            try:
                user_question = input("\nAsk a question about the maps: ").strip()
                
                if user_question.lower() in ["exit", "quit", "q"]:
                    print("Exiting the chat...Bye Bye!")
                    break
                
                if not user_question:
                    continue
                
                answer = self.rag_generator.generate_answer(user_question)
                print("\nAnswer:", answer)
            
            except KeyboardInterrupt:
                print("\nChat interrupted. Type 'exit' to quit.")
            except Exception as e:
                print(f"An error occurred: {e}")

In [12]:
def main():
    my_dir = r'C:\Users\KIIT0001\Desktop\RAG Chatbot\my_dir'
    if not os.path.exists(my_dir):
        print(f"Error: Directory {my_dir} does not exist.")
        return

    pipeline = RAGChatPipeline(my_dir)
    pipeline.run_chat()

if __name__ == "__main__":
    main()

Starting the RAG Chat System...
Type 'exit' to quit the chat.

Answer: City Hall (Node A) is located at the intersection of Main Street, Secondary Road, and the pathway to the gymnasium.


Sources:
- map_descriptions.txt
- map_descriptions.txt
- map_descriptions.txt

Answer: The locations mentioned in the text are: City Hall (Node A), Central Hospital (Node B), Fire Station (Node C), Subway Entrance (Node D), Apartment Complex (Node E), School Gymnasium (Node F), Library Basement (Node G), Major Intersection 1 (Node H), Major Intersection 2 (Node I), Park Area (Node J), Campsite (Node K), Cliffside Lookout (Node L), Small Residential Block (Node M), Riverside Path (Node N), Collapsed Bridge (Node O), and a Riverside Pathway (Node N).

Additionally, there are Safe Zones at the School Gymnasium (Node F) and Library Basement (Node G), an Obstacle at the Collapsed Bridge (Node O), and High-Density Areas around Nodes A, B, C, and the surrounding intersections.



Sources:
- map_descriptions